In [1]:
import torch
import torch.nn as nn 
from torch.nn import functional as F
import mmap
import random
import pickle

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# print(device)

block_size = 32
batch_size = 128
learning_rate = 3e-4
max_iters = 200
eval_iters = 100
dropout = 0.2
n_embd = 384
n_layer= 1
n_head = 1

print(device)

cuda


In [5]:
with open("openwebtext/vocab.txt", 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
vocab_size = len(chars)
vocab_size

32172

In [7]:
string_to_text=  {ch:i for i , ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_text[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [9]:
Q = 1
def get_random_chunk(split):
    filename = "openwebtext/output_train.txt" if split == "train" else "openwebtext/output_val.txt"
    with open(filename, "rb") as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            file_size = len(mm)
            start_pos = random.randint(0 , (file_size) - Q*block_size*batch_size)

            #seek to random pos
            mm.seek(start_pos)
            block = mm.read(Q*block_size*batch_size-1)

            #decode binary
            decoded_block = block.decode("utf-8", errors="ignore").replace("\r","")

            #train test splts
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
    return data
    

In [11]:
def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size, )) #take random index from 0 to block size
    x = torch.stack([data[i : i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device) ,y.to(device)
    return x, y


In [13]:
@torch.no_grad()
def estimate_loss():
    out= {}
    model.eval()
    for split in ['train', 'val']:
        losses= torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [19]:
class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float("-inf"))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out
    
class  MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd) ## 
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) - > (B, T, [h1, h2, h3])
        out = self.dropout(self.proj(out))
        return out


class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd), 
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self,x ):
        return self.net(x)
        
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size=  n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        #FF-NN
        self.ffwd = FeedForward(n_embd)
        #add + norm, post-norm architecture /\ (X + Y) , Y -> F(X)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x+y)
        y = self.ffwd(x)
        x = self.ln2(x+y)
        return x
        
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd) #positional encoding
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)]) #hidden D-layer(s)
        self.ln_f = nn.LayerNorm(n_embd) #layer normalisation
        self.lm_head = nn.Linear(n_embd, vocab_size) #linear transformation

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    def forward(self, index, targets=None):
        # B, T, C = logits.shape
        B , T = index.shape

        tok_emb = self.token_embedding_table(index) # B , T , C
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # T,C 
        x= tok_emb + pos_emb # B, T, C
        x = self.blocks(x) # B, T, C
        x = self.ln_f(x) # B, T,C 
        logits = self.lm_head(x) # B, T, VOCAB_SIZE

        if targets is None:
            loss = None
        else:
            #BATCH, TIME / SEQ LENGTH, VOCAB SIZE / CHANNEL
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets= targets.view(B*T)
            loss = F.cross_entropy(logits, targets) 

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[: , - 1, :] #becomes (B,C), last time sample
            probs = F.softmax(logits, dim=-1) #(B, C) where vocab dimension=[:-1]
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1 )
            index = torch.cat((index, index_next), dim=1) # (B, T+1)

        return index

    def getEmbedding(self):
        return self.token_embedding_table

model = GPTLanguageModel(vocab_size)
print("loading model parameters...")
with open('model-01.pkl' ,'rb') as f:
    model = pickle.load()
print("load successfully")
m = model.to(device)
    

# context = torch.zeros((1,1), dtype=torch.long, device=device) 
# generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
# print(generated_chars)
# generated_chars_raw = m.generate(context, max_new_tokens=500)
# print(generated_chars_raw)



loading model parameters...


TypeError: load() missing required argument 'file' (pos 1)

In [21]:
optimiser = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    # print(iter)
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter} , train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    #sample batch data
    xb, yb = get_batch('train') #in, ot

    #eval loss
    logits, loss = model.forward(xb, yb)
    optimiser.zero_grad(set_to_none = True) #enable for RNNs
    loss.backward()
    optimiser.step()
print(loss.item())


with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device) #(1,1) being new space char, starting context
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)